# 1. Preprocess Data

In [ ]:
!pip install gdown
!pip install huggingface_hub

In [ ]:
import gdown

url = "https://drive.google.com/drive/folders/1KLzX-PB_1SeufhlLrrGrArDw21F2KMgT"
gdown.download_folder(url, quiet=True, use_cookies=False)

In [13]:
%cd manual_reviewed
!dir
!rm batch1
!mv batch1s batch1
!dir

batch1	batch10  batch2  batch3  batch4  batch5  batch6  batch7  batch8  batch9


In [31]:
import pandas as pd

df = pd.DataFrame()

for i in range(10):
    inp = pd.read_excel(f'batch{i+1}', usecols=['input', 'output', 'instruction'])
    df = pd.concat([df, inp])

df = df.reset_index(drop=True)[['input', 'output', 'instruction']]
df

,input,output,instruction
0,"Dokter, aku mengalami serangan panik tiba-tiba...","Berdasarkan apa yang Anda katakan, sepertinya ...","Jika Anda seorang dokter, silakan menjawab per..."
1,"Dokter, aku sudah beberapa minggu ini mengalam...",Anda mungkin memiliki polip tali suara. Untuk ...,"Jika Anda seorang dokter, silakan menjawab per..."
2,"Halo, Dokter, aku mungkin memiliki sindrom Tur...","Oke, dalam hal itu, Anda perlu menjalani pemer...","Jika Anda seorang dokter, silakan menjawab per..."
3,"Dokter, aku mengalami kondisi yang disebut kri...",Anda harus menjalani pemeriksaan kulit fisik l...,"Jika Anda seorang dokter, silakan menjawab per..."
4,"Dokter, aku pikir aku telah diracuni. Aku tida...",Maaf mendengar itu. Kita perlu melakukan beber...,"Jika Anda seorang dokter, silakan menjawab per..."
...,...,...,...
4995,"Dokter, aku sering buang air kecil, gatal vagi...","Berdasarkan gejala Anda, tampaknya Anda mungki...","Jika Anda seorang dokter, silakan menjawab per..."
4996,Aku punya banyak masalah dengan buang air keci...,Gejala ini bisa disebabkan oleh Human Papillom...,"Jika Anda seorang dokter, silakan menjawab per..."
4997,"Dokter, aku mengalami buang air kecil, buang a...","Berdasarkan gejala Anda, kemungkinan Anda memi...","Jika Anda seorang dokter, silakan menjawab per..."
4998,"Hei dokter, aku mengalami sakit lengan, sakit ...","Berdasarkan gejala Anda, mungkin Anda memiliki...","Jika Anda seorang dokter, silakan menjawab per..."


In [33]:
for i in df.index[:5]:
    print(df.loc[i].to_json())
    print(type(df.loc[i].to_json()))

{"input":"Dokter, aku mengalami serangan panik tiba-tiba dan sering.","output":"Berdasarkan apa yang Anda katakan, sepertinya Anda mungkin menderita gangguan panik. Langkah terbaik adalah memulai dengan psikoterapi dan konseling kesehatan mental. Selain itu, kita harus melakukan elektrokardiogram untuk memastikan bahwa tidak ada masalah fisik yang menyebabkan serangan panik Anda. Kita juga perlu melakukan skrining depresi dan skrining toksikologi untuk mengesampingkan penyebab lain. Akhirnya, saya akan merekomendasikan evaluasi dan terapi psikologis dan psikiatri yang komprehensif untuk membantu mengelola gejala Anda.","instruction":"Jika Anda seorang dokter, silakan menjawab pertanyaan medis berdasarkan deskripsi pasien."}
<class 'str'>
{"input":"Dokter, aku sudah beberapa minggu ini mengalami suara serak dan tidak kunjung membaik meskipun sudah minum obat. Apa masalahnya?","output":"Anda mungkin memiliki polip tali suara. Untuk mengkonfirmasi ini, kita perlu melakukan trakeoskopi dan

In [36]:
with open('allBatch.json', 'w') as f:
    for i in df.index:
        f.writelines(df.loc[i].to_json() + '\n')

In [ ]:
from datasets import load_dataset

# Adjust with your path
data = load_dataset("json", data_files='/kaggle/input/allbatch/allBatch.json')
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ca991695af4ec410/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 5000
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# 2. LlaMa 2

In [ ]:
!git clone https://github.com/tloen/alpaca-lora
!pip install -r alpaca-lora/requirements.txt

In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
base_model = 'meta-llama/Llama-2-7b-hf'
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = LlamaTokenizer.from_pretrained(base_model)
tokenizer.pad_token_id = (0)  # unk. we want this to be different from the eos token
tokenizer.padding_side = "left"  # Allow batched inference

In [6]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training
)

In [7]:
model = prepare_model_for_int8_training(model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [8]:
# Lora parameters
lora_r = 8
lora_alpha = 16
lora_target_modules = ["q_proj", "v_proj"]
lora_dropout = 0.05

config = LoraConfig(
  r=lora_r,
  lora_alpha=lora_alpha,
  target_modules=lora_target_modules,
  lora_dropout=lora_dropout,
  bias="none",
  task_type="CAUSAL_LM",
)

In [9]:
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [6]:
def generate_prompt(
    instruction, input, label
):
  template = {
      "prompt_input": "Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan masukan yang memberikan konteks lebih lanjut. Tulis tanggapan yang melengkapi permintaan dengan tepat.\n\n### Instruksi:\n{instruction}\n\n### Masukan:\n{input}",
      "response_split": "### Tanggapan:"
  }

  if label:
      res = '''<s>[INST] <<SYS>>\n{0}\n<</SYS>>\n\n{1} [/INST] {2} </s>'''.format(template['prompt_input'].format(instruction=instruction, input=input), template['response_split'], label)
  else:
      res = '''<s>[INST] <<SYS>>\n{0}\n<</SYS>>\n\n{1} [/INST]'''.format(template['prompt_input'].format(instruction=instruction, input=input), template['response_split'])
      
  return res

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )

    cutoff_len = 256
    result = tokenizer(
        full_prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )

    if (result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < cutoff_len):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    tokenized_full_prompt = result


    return tokenized_full_prompt

In [7]:
train_val = data["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt)) #Set to 500 out from 5000
train_data

  0%|          | 0/4500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

Dataset({
    features: ['input', 'output', 'instruction', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4500
})

In [17]:
batch_size = 128
micro_batch_size = 4
gradient_accumulation_steps = batch_size / micro_batch_size
num_epochs = 3
learning_rate = 3e-4
val_set_size = 2
output_dir = './result'
group_by_length = False

trainer = Trainer(
  model=model,
  train_dataset=train_data,
  eval_dataset=val_data,
  args=TrainingArguments(
      per_device_train_batch_size=micro_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      warmup_steps=100,
      num_train_epochs=num_epochs,
      learning_rate=learning_rate,
      fp16=True,
      logging_steps=10,
      optim="adamw_torch",
      evaluation_strategy="steps" if val_set_size > 0 else "no",
      save_strategy="steps",
      eval_steps=200 if val_set_size > 0 else None,
      save_steps=200,
      output_dir=output_dir,
      save_total_limit=3,
      load_best_model_at_end=True if val_set_size > 0 else False,
      ddp_find_unused_parameters=None,
      group_by_length=group_by_length
),
  data_collator=DataCollatorForSeq2Seq(
      tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
  ),
)

In [18]:
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(model, type(model))

In [ ]:
model = torch.compile(model)
trainer.train()

In [21]:
model.push_to_hub('Llama-2-7b-finetuned-2', safe_serialization=True)

adapter_model.safetensors:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fadliaulawi/Llama-2-7b-finetuned-2/commit/65144b5fae005bb987602fa9a6cf9102284a7271', commit_message='Upload model', commit_description='', oid='65144b5fae005bb987602fa9a6cf9102284a7271', pr_url=None, pr_revision=None, pr_num=None)

# PolyLM

In [ ]:
!git clone https://github.com/tloen/alpaca-lora
!pip install -r alpaca-lora/requirements.txt

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

base_model = 'DAMO-NLP-MT/polylm-1.7b'
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
tokenizer.pad_token_id = (0)  # unk. we want this to be different from the eos token
tokenizer.padding_side = "left"  # Allow batched inference

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training
)

model = prepare_model_for_int8_training(model)

# Lora parameters
lora_r = 8
lora_alpha = 16
lora_target_modules = ["q_proj", "v_proj"]
lora_dropout = 0.05

config = LoraConfig(
  r=lora_r,
  lora_alpha=lora_alpha,
  lora_dropout=lora_dropout,
  bias="none",
  task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,738,657,792 || trainable%: 0.09046426543723217


/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
def generate_prompt(
    instruction, input, label
):
  template = {
      "prompt_input": "Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan masukan yang memberikan konteks lebih lanjut. Tulis tanggapan yang melengkapi permintaan dengan tepat.\n\n### Instruksi:\n{instruction}\n\n### Masukan:\n{input}",
      "response_split": "### Tanggapan:"
  }

  res = template["prompt_input"].format(instruction=instruction, input=input)

  if label:
      res = f"{res} \n\n### Tanggapan:\n{label}"

  return res

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )

    cutoff_len = 256
    result = tokenizer(
        full_prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )

    if (result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < cutoff_len):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    tokenized_full_prompt = result


    return tokenized_full_prompt

In [ ]:
train_val = data["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt)) #Set to 500 out from 5000
train_data

  0%|          | 0/4500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

Dataset({
    features: ['input', 'output', 'instruction', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4500
})

In [10]:
batch_size = 128
micro_batch_size = 4
gradient_accumulation_steps = batch_size / micro_batch_size
num_epochs = 3
learning_rate = 3e-4
val_set_size = 2
output_dir = './result'
group_by_length = False

trainer = Trainer(
  model=model,
  train_dataset=train_data,
  eval_dataset=val_data,
  args=TrainingArguments(
      per_device_train_batch_size=micro_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      warmup_steps=100,
      num_train_epochs=num_epochs,
      learning_rate=learning_rate,
      fp16=True,
      logging_steps=10,
      optim="adamw_torch",
      evaluation_strategy="steps" if val_set_size > 0 else "no",
      save_strategy="steps",
      eval_steps=200 if val_set_size > 0 else None,
      save_steps=200,
      output_dir=output_dir,
      save_total_limit=3,
      load_best_model_at_end=True if val_set_size > 0 else False,
      ddp_find_unused_parameters=None,
      group_by_length=group_by_length
),
  data_collator=DataCollatorForSeq2Seq(
      tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
  ),
)

In [ ]:
model = torch.compile(model)
trainer.train()

In [14]:
model.push_to_hub('polylm-1.7b-finetuned')

adapter_model.bin:   0%|          | 0.00/6.31M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fadliaulawi/polylm-1.7b-finetuned/commit/5385485e8e489c405600862be285673f51cc3f0b', commit_message='Upload model', commit_description='', oid='5385485e8e489c405600862be285673f51cc3f0b', pr_url=None, pr_revision=None, pr_num=None)

# InternLM

In [ ]:
!git clone https://github.com/tloen/alpaca-lora
!pip install -r alpaca-lora/requirements.txt

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

base_model = 'internlm/internlm-7b'
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


A new version of the following files was downloaded from https://huggingface.co/internlm/internlm-7b:
- configuration_internlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/internlm/internlm-7b:
- modeling_internlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token_id = (0)  # unk. we want this to be different from the eos token
tokenizer.padding_side = "left"  # Allow batched inference

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm-7b:
- tokenization_internlm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [4]:
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_int8_training
)

model = prepare_model_for_int8_training(model)

# Lora parameters
lora_r = 8
lora_alpha = 16
lora_target_modules = ["q_proj", "v_proj"]
lora_dropout = 0.05

config = LoraConfig(
  r=lora_r,
  lora_alpha=lora_alpha,
  target_modules=lora_target_modules,
  lora_dropout=lora_dropout,
  bias="none",
  task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 4,194,304 || all params: 7,326,142,464 || trainable%: 0.05725119352524783


In [ ]:
def generate_prompt(
    instruction, input, label
):
  template = {
      "prompt_input": "Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan masukan yang memberikan konteks lebih lanjut. Tulis tanggapan yang melengkapi permintaan dengan tepat.\n\n### Instruksi:\n{instruction}\n\n### Masukan:\n{input}",
      "response_split": "### Tanggapan:"
  }

  res = template["prompt_input"].format(instruction=instruction, input=input)

  if label:
      res = f"{res} \n\n### Tanggapan:\n{label}"

  return res

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )

    cutoff_len = 256
    result = tokenizer(
        full_prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )

    if (result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < cutoff_len):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    tokenized_full_prompt = result


    return tokenized_full_prompt

In [ ]:
train_val = data["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt)) #Set to 500 out from 5000
train_data

  0%|          | 0/4500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

Dataset({
    features: ['input', 'output', 'instruction', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4500
})

In [8]:
batch_size = 128
micro_batch_size = 4
gradient_accumulation_steps = batch_size / micro_batch_size
num_epochs = 3
learning_rate = 3e-4
val_set_size = 2
output_dir = './result'
group_by_length = False

trainer = Trainer(
  model=model,
  train_dataset=train_data,
  eval_dataset=val_data,
  args=TrainingArguments(
      per_device_train_batch_size=micro_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      warmup_steps=100,
      num_train_epochs=num_epochs,
      learning_rate=learning_rate,
      fp16=True,
      logging_steps=10,
      optim="adamw_torch",
      evaluation_strategy="steps" if val_set_size > 0 else "no",
      save_strategy="steps",
      eval_steps=200 if val_set_size > 0 else None,
      save_steps=200,
      output_dir=output_dir,
      save_total_limit=3,
      load_best_model_at_end=True if val_set_size > 0 else False,
      ddp_find_unused_parameters=None,
      group_by_length=group_by_length
),
  data_collator=DataCollatorForSeq2Seq(
      tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
  ),
)

In [ ]:
model = torch.compile(model)
trainer.train()

In [12]:
model.push_to_hub('internlm-7b-finetuned')

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fadliaulawi/internlm-7b-finetuned/commit/92d245cd0ed8e7b020a2ac2d53d102cbfe4b242b', commit_message='Upload model', commit_description='', oid='92d245cd0ed8e7b020a2ac2d53d102cbfe4b242b', pr_url=None, pr_revision=None, pr_num=None)